## Первый проход

### Установка зависимостей

In [8]:
from transformers import pipeline
import time
import torch
import textwrap
import numpy as np
import pandas as pd
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

### Считывание Excel

In [ ]:
file_path = 'Data.xlsx' # No password file
xlsx = pd.ExcelFile(file_path)

print(xlsx.sheet_names)

employee_responses = pd.read_excel(xlsx, sheet_name='ответы сотрудников')
hr_responses = pd.read_excel(xlsx, sheet_name='ответы hr ')

employee_responses.head()
# hr_responses.head()

### Использование pipeline summarization

In [35]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
summarizer = pipeline("summarization", model="google/mt5-large", device=device)
# summarizer = pipeline("summarization", model="t5-small", device=device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [40]:
text = """
intelligence demonstrated by machines in contrast to the natural intelligence displayed by humans and animals leading ai textbooks define the field
"""

summary = summarizer(text, max_length=20, min_length=2, do_sample=False)

print(summary[0]['summary_text'])

<extra_id_0> of intelligence leading to intelligence displayed by machines and animals a


### Pegasus

In [25]:
from transformers import PegasusForConditionalGeneration, AutoTokenizer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'google/pegasus-cnn_dailymail'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import MarianMTModel, MarianTokenizer

ru_en_model_name = "Helsinki-NLP/opus-mt-ru-en"
en_ru_model_name = "Helsinki-NLP/opus-mt-en-ru"

ru_en_tokenizer = MarianTokenizer.from_pretrained(ru_en_model_name)
ru_en_model = MarianMTModel.from_pretrained(ru_en_model_name).to(device)

en_ru_tokenizer = MarianTokenizer.from_pretrained(en_ru_model_name)
en_ru_model = MarianMTModel.from_pretrained(en_ru_model_name).to(device)

c:\Users\alex\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\alex\anaconda3\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [13]:
def translate(text, model, tokenizer, device):
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    translated_ids = model.generate(inputs["input_ids"])
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translated_text

In [34]:
# Исходный текст на русском языке
ru_text = "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу."

# Перевод с русского на английский
en_text = translate(ru_text, ru_en_model, ru_en_tokenizer, device)

# Текст для суммаризации (на английском)
print("Translated to English:", en_text)

# Токенизация текста для PEGASUS
inputs = tokenizer(en_text, return_tensors="pt", max_length=512, truncation=True).to(device)

# Генерация суммаризации
summary_ids = model.generate(inputs["input_ids"], max_length=60, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

# Декодирование результата
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Summary in English:", summary)

# Перевод суммаризации обратно на русский
ru_summary = translate(summary, en_ru_model, en_ru_tokenizer, device)

print("Summary in Russian:", ru_summary)

Translated to English: Life's difficulties contributed to my dismissal because I had to go to another job.
Summary in English: Life's difficulties contributed to my dismissal because I had to go to another job.<n>I had to go to another job because I had to go to another job.<n>I was fired because I had to go to another job.
Summary in Russian: Мне пришлось пойти на другую работу, потому что мне пришлось пойти на другую работу.


In [1]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

c:\Users\alex\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
# article_text = "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу"
article_text = "Руководство регулярно манипулирует, газлайтит (виноват всегда и во всем сотрудник и ты начинаешь верить, что ты бездарь), токсичное отношение к сотрудникам."

input_ids = tokenizer(
    [article_text],
    max_length=15,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

Westphal Corporation регулярно манипулирует руководством, газлайтит, манипулирует.


In [22]:
def summarize_answer(text):
    summary = summarizer(text, max_length=10, min_length=2, do_sample=False)[0]['summary_text']
    return summary.strip()

# answers = [
#     "в связи с переездом в другой регион,семейные обстоятельства",
#     "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу",
#     "Руководство регулярно манипулирует, газлайтит (виноват всегда и во всем сотрудник и ты начинаешь верить, что ты бездарь), токсичное отношение к сотрудникам." 
# ]

answers = [
   "due to moving to another region, family circumstances",
   "Life difficulties contributed to my dismissal, since I need to go to another job",
   "Management regularly manipulates, gaslights (the employee is always to blame for everything and you begin to believe that you are a loser), toxic attitude towards employees."
]

for answer in answers:
  short_answer = summarize_answer(answer)
  print(f"Исходный ответ: {answer}")
  print(f"Суммированный ответ: {short_answer}\n")

Исходный ответ: due to moving to another region, family circumstances
Суммированный ответ: . due to moving to

Исходный ответ: Life difficulties contributed to my dismissal, since I need to go to another job
Суммированный ответ: "Life difficulties contributed to my dismissal

Исходный ответ: Management regularly manipulates, gaslights (the employee is always to blame for everything and you begin to believe that you are a loser), toxic attitude towards employees.
Суммированный ответ: Management regularly manipulates, gaslights



## Тестирование моделей

### Тестирование через pipeline

In [ ]:
models = [
   ("BART", "facebook/mbart-large-cc25cnn"),
   ("AIForever", 'ai-forever/ru-en-RoSBERTa'),
   ("SberBert", "sberbank-ai/ruBert-base"),
   ("DeepPavlovBert", "DeepPavlov/rubert-base-cased-conversational"),
   ("IlyaGusev", "IlyaGusev/mbart_ru_sum_gazeta")
]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def summarize_answer(prompt, 
                     text, 
                     model_name,
                     model_path):
    summarizer = pipeline("summarization", model=model_path, device=device)
    summary = summarizer(prompt + text, max_length=15, min_length=1, do_sample=False)
    print (summary)

answers = [
    "Мой карьерный рост слишком сильно отстал по отношению к тому функционалу и той ответственности, которые я на себя брал. Многие мои коллеги из ЦАТЭ, обладая гораздо меньшим опытом, оказались на несколько должностей выше меня. Как результат, после очередного выгорания на рабочем месте, мною был запущен мониторинг рынка труда, который показал, что меня на данный момент многие компании оценивают на уровне глав.спеца (при моей 2 категории сейчас). К величайшему сожалению, моя замечательная начальница уже не имела и малейшего шанса выправить ситуацию, хоть и приложила максимум усилий для этого. Если бы компания сумела предложить мне то, что предлагают другие участники рынка труда - я бы остался, так как здесь у меня сложились очень теплые отношения со многими коллегами, и мне очень жаль с ними расставаться. Но у меня есть ответственность за свою семью. Потому я буду исходить именно из ее интересов. Я бы мог еще многое рассказать. О глубоких проблемах АТЭ и данного дивизиона. Однако там материала минимум на пару часов, сюда писать смысла нет. Потому пожелаю удачи всем тем, кто вместе со мной пережил немало трудностей в данной компании. Спасибо вам, персонал ЦАТЭ!",
    "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу",
    "Руководство регулярно манипулирует, газлайтит (виноват всегда и во всем сотрудник и ты начинаешь верить, что ты бездарь), токсичное отношение к сотрудникам. Похвалили, сказали, что переведут на другую должность+оклад поднимут почти в 2 раза, по факту обвинили в том, что что-то не было сделано (хотя задач таких никто не ставил). Посыл: работай больше, ты и так не справляешься, поднимать зп не будем. Разброс зп в отделе от 130 тыс. до 60!!! КАК можно платить сотруднику 60 тыс. в Москве?!" 
]

reference_summary = "Переезд, семейные обстоятельства, токсичное руководство"
# prompt = "Сократи текст до 2-3 слов, передав суть:"
prompt = ""

for answer in answers:
  print(f"Исходный ответ: {answer}")
  for model_name, model_path in models:
    summary = summarize_answer(prompt, answer, model_name, model_path)

In [ ]:
def summarize_text(model_name, text, max_length=10, num_beams=4):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    input_text = "summarize: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    summary_ids = model.generate(input_ids, max_length=max_length, num_beams=num_beams, early_stopping=True)
    
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

model_name = [
   ("BART", "facebook/mbart-large-cc25cnn"),
   ("AIForever", 'ai-forever/ru-en-RoSBERTa'),
   ("SberBert", "sberbank-ai/ruBert-base"),
   ("DeepPavlovBert", "DeepPavlov/rubert-base-cased-conversational"),
   ("IlyaGusev", "IlyaGusev/mbart_ru_sum_gazeta")
]

text = """
    Этот ответ подробно объясняет процесс работы пользователя, как он подошел к решению проблемы и его аргументацию.
    Включены детали о различных шагах, которые были предприняты, проблемы, с которыми он столкнулся, и возможные решения.
    Также приводятся идеи, как можно улучшить процесс и оптимизировать задачу.
"""

for model_name, model_path in models.items():
    print(f"\n--- {model_name} ---")
    summary = summarize_text(model_path, text)
    print("Summarized text:", summary)

In [41]:
from transformers import pipeline

# Создаем пайплайн для суммаризации
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
summarizer = pipeline("summarization", model="IlyaGusev/mbart_ru_sum_gazeta", device=device)

# Пример текста на русском
text = "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу"

# Выполняем суммаризацию
summary = summarizer(text, max_length=13, min_length=2, do_sample=False)

# Печатаем результат
print(summary[0]['summary_text'])

В связи с многочисленными проблемами со здоровьем


### Тяжелые локальные модели

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Загрузка модели mT5 (большая модель для суммаризации)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели mT5 (большая модель для суммаризации)
tokenizer = AutoTokenizer.from_pretrained("google/mt5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-large").to(device)

c:\Users\alex\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\alex\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In

In [18]:
# text = "Жизненные трудности поспособствовали моему увольнению, так как нужно идти на другую работу."
# text = "Руководство регулярно манипулирует, газлайтит (виноват всегда и во всем сотрудник и ты начинаешь верить, что ты бездарь), токсичное отношение к сотрудникам. Похвалили, сказали, что переведут на другую должность+оклад поднимут почти в 2 раза, по факту обвинили в том, что что-то не было сделано (хотя задач таких никто не ставил). Посыл: работай больше, ты и так не справляешься, поднимать зп не будем. Разброс зп в отделе от 130 тыс. до 60!!! КАК можно платить сотруднику 60 тыс. в Москве?!"
text = "Перескажи в 2-3 слова следующий текст: Был необходим отпуск на длительное время (3 мес) из-за семейных обстоятельств. Руководитель отказал."
# text = """Compress to 2-3 words: intelligence demonstrated by machines in contrast to the natural intelligence displayed by humans and animals leading ai textbooks define the field"""

# Translate
text = translate(text, ru_en_model, ru_en_tokenizer, device)
print(text)

# Токенизация текста
start_time = time.time()
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
inputs = {key: value.to(device) for key, value in inputs.items()}
print("Time for tokenizer: ", time.time() - start_time)

# Настройка параметров генерации (выход - 10 токенов, что приближенно к 2-3 словам)

start_time = time.time()
summary_ids = model.generate(inputs["input_ids"])
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
summary_words = " ".join(summary.split())
print("Time for generation: ", time.time() - start_time)

# Translate
summary_words = translate(summary_words, en_ru_model, en_ru_tokenizer, device)

print(summary_words)

Say two or three words: a long leave was needed (3 months) because of family circumstances.
Time for tokenizer:  0.0009992122650146484
Time for generation:  0.8217849731445312
<extra_id_0> требуется продолжительный отпуск (3 месяца) из-за семейных обстоятельств.


In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
import gc
gc.collect()
torch.cuda.empty_cache()